# Description

This notebook reads a PR from a manuscript and matches original paragraphs with modified ones.

# Modules

In [1]:
from pathlib import Path

import pandas as pd
from github import Auth, Github
from IPython.display import display
from proj import conf
from proj.utils import process_paragraph

# Settings/paths

In [2]:
REPO = "pivlab/manubot-ai-editor-code-test-biochatter-manuscript"
PR = (2, "gpt-3.5-turbo")
# PR = (3, "gpt-4-0125-preview")

OUTPUT_FILE_PATH = None

In [3]:
# Parameters
OUTPUT_FILE_PATH = "/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/paragraph_match/biochatter-manuscript--gpt-3.5-turbo.pkl"


In [4]:
OUTPUT_FILE_PATH = Path(OUTPUT_FILE_PATH).resolve()
OUTPUT_FILE_PATH.parent.mkdir(parents=True, exist_ok=True)
display(OUTPUT_FILE_PATH)

PosixPath('/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/paragraph_match/biochatter-manuscript--gpt-3.5-turbo.pkl')

# Get Repo

In [5]:
auth = Auth.Token(conf.github.API_TOKEN)

In [6]:
g = Github(auth=auth)

In [7]:
repo = g.get_repo(REPO)

# Get Pull Request

In [8]:
pr = repo.get_pull(PR[0])

In [9]:
list(pr.get_files())

[File(sha="35c709fec248ca8fbc5c4254294c126da6879707", filename="content/01.abstract.md"),
 File(sha="d2b401a7c0b5352b4f8299fe7822e46f67723289", filename="content/10.introduction.md"),
 File(sha="fcdb707a8ee8f2b88f2e385bfa65081e6a96d05b", filename="content/20.results.md"),
 File(sha="df7cb00d842d36d076440ddabed093974e9a14b5", filename="content/30.discussion.md"),
 File(sha="9ec3ba88154efcb02dae746e91e93141c97df67e", filename="content/40.methods.md")]

In [10]:
pr_commits = list(pr.get_commits())

In [11]:
pr_commits[0].parents

[Commit(sha="38a0c074e5eed7da6351cda324458e6f4ed4ece4")]

In [12]:
pr_prev = pr_commits[0].parents[0].sha
print(pr_prev)

38a0c074e5eed7da6351cda324458e6f4ed4ece4


In [13]:
pr_curr = pr_commits[0].sha
print(pr_curr)

fd1a546a487e1cb894d9cf11c9b873735d5f72a3


# Get file list

In [14]:
pr_files = [f for f in pr.get_files() if f.filename.endswith(".md")]
display(pr_files)

[File(sha="35c709fec248ca8fbc5c4254294c126da6879707", filename="content/01.abstract.md"),
 File(sha="d2b401a7c0b5352b4f8299fe7822e46f67723289", filename="content/10.introduction.md"),
 File(sha="fcdb707a8ee8f2b88f2e385bfa65081e6a96d05b", filename="content/20.results.md"),
 File(sha="df7cb00d842d36d076440ddabed093974e9a14b5", filename="content/30.discussion.md"),
 File(sha="9ec3ba88154efcb02dae746e91e93141c97df67e", filename="content/40.methods.md")]

# Sections

In [15]:
# TODO: explain variable
paragraph_matches = []

## Abstract

In [16]:
section_name = "abstract"

In [17]:
pr_filename = pr_files[0].filename
assert section_name in pr_filename
print(pr_filename)

content/01.abstract.md


### Original

In [18]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## Abstract

Current-generation Large Language Mod


In [19]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

2

### Modified

In [20]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## Abstract

Large Language Models (LLMs) have gen


In [21]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

2

### Match

In [22]:
orig_section_paragraphs[0]

'## Abstract'

In [23]:
mod_section_paragraphs[0]

'## Abstract'

####  Paragraph 00

In [24]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

Current-generation Large Language Models (LLMs) have stirred enormous interest in recent months, yielding great potential for accessibility and automation, while simultaneously posing significant challenges and risk of misuse. To facilitate interfacing with LLMs in the biomedical space, while at the same time safeguarding their functionalities through sensible constraints, we propose a dedicated, open-source framework: BioChatter. Based on open-source software packages, we synergise the many functionalities that are currently developing around LLMs, such as knowledge integration / retrieval-augmented generation, model chaining, and benchmarking, resulting in an easy-to-use and inclusive framework for application in many use cases of biomedicine. We focus on robust and user-friendly implementation, including ways to deploy privacy-preserving local open-source LLMs. We demonstrate use cases via two multi-purpose web apps ([https://chat.biocypher.org](https://chat.biocypher.org)), and pro

In [25]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

Large Language Models (LLMs) have generated significant interest due to their potential for accessibility and automation in various fields, including biomedicine. However, they also present challenges and risks of misuse. In this paper, we address the need for a framework to interface with LLMs in the biomedical domain while ensuring their safe and effective use. To meet this need, we introduce BioChatter, an open-source framework that integrates various functionalities of LLMs, such as knowledge integration, retrieval-augmented generation, model chaining, and benchmarking. By leveraging open-source software packages, we have developed a user-friendly and versatile platform that can be applied across a range of biomedicine use cases. Our focus is on implementing robust and privacy-preserving local open-source LLMs. We showcase the utility of BioChatter through two multi-purpose web apps available at [https://chat.biocypher.org](https://chat.biocypher.org) and provide comprehensive docu

In [26]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [27]:
display(paragraph_matches[-1])

('abstract',
 'Current-generation Large Language Models (LLMs) have stirred enormous interest in recent months, yielding great potential for accessibility and automation, while simultaneously posing significant challenges and risk of misuse. To facilitate interfacing with LLMs in the biomedical space, while at the same time safeguarding their functionalities through sensible constraints, we propose a dedicated, open-source framework: BioChatter. Based on open-source software packages, we synergise the many functionalities that are currently developing around LLMs, such as knowledge integration / retrieval-augmented generation, model chaining, and benchmarking, resulting in an easy-to-use and inclusive framework for application in many use cases of biomedicine. We focus on robust and user-friendly implementation, including ways to deploy privacy-preserving local open-source LLMs. We demonstrate use cases via two multi-purpose web apps ([https://chat.biocypher.org](https://chat.biocypher

## Introduction

In [28]:
section_name = "introduction"

In [29]:
pr_filename = pr_files[1].filename
assert section_name in pr_filename
print(pr_filename)

content/10.introduction.md


### Original

In [30]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## Introduction

Despite technological advances, u


In [31]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

5

### Modified

In [32]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## Introduction

Despite technological advances, u


In [33]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

7

### Match

In [34]:
orig_section_paragraphs[0]

'## Introduction'

In [35]:
mod_section_paragraphs[0]

'## Introduction'

####  Paragraph 00

In [36]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

Despite technological advances, understanding biological and biomedical systems still poses major challenges [@gallagher-infinite;@dl-bioscience]. We measure more and more data points with ever-increasing resolution to such a degree that their analysis and interpretation have become the bottleneck for their exploitation [@dl-bioscience]. One reason for this challenge may be the inherent limitation of human knowledge [@doi:10.1016/j.tics.2005.04.010]: Even seasoned domain experts cannot know the implications of every gene, molecule, symptom, or biomarker. In addition, biological events are context-dependent, for instance with respect to a cell type or specific disease.


In [37]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

Despite technological advances, understanding biological and biomedical systems continues to present significant challenges (Gallagher et al., 2020; DL Bioscience, 2019). The volume of data generated is increasing exponentially, leading to a bottleneck in the analysis and interpretation of this data (DL Bioscience, 2019). One possible explanation for this challenge is the inherent limitation of human knowledge (Smith, 2005). Even experts in the field may not fully comprehend the implications of every gene, molecule, symptom, or biomarker. Moreover, biological processes are influenced by various contextual factors, such as cell type or specific disease conditions.


In [38]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [39]:
display(paragraph_matches[-1])

('introduction',
 'Despite technological advances, understanding biological and biomedical systems still poses major challenges [@gallagher-infinite;@dl-bioscience]. We measure more and more data points with ever-increasing resolution to such a degree that their analysis and interpretation have become the bottleneck for their exploitation [@dl-bioscience]. One reason for this challenge may be the inherent limitation of human knowledge [@doi:10.1016/j.tics.2005.04.010]: Even seasoned domain experts cannot know the implications of every gene, molecule, symptom, or biomarker. In addition, biological events are context-dependent, for instance with respect to a cell type or specific disease.',
 'Despite technological advances, understanding biological and biomedical systems continues to present significant challenges (Gallagher et al., 2020; DL Bioscience, 2019). The volume of data generated is increasing exponentially, leading to a bottleneck in the analysis and interpretation of this data

####  Paragraph 01

In [40]:
par0 = process_paragraph(orig_section_paragraphs[2])
print(par0)

Large Language Models (LLMs) of the current generation, in contrast, can access enormous amounts of knowledge, encoded (incomprehensibly) in their billions of parameters [@doi:10.48550/arxiv.2204.02311;@doi:10.48550/arxiv.2201.08239;@doi:10.48550/arxiv.2303.08774;@doi:10.1609/aaai.v36i11.21488]. Trained correctly, they can recall and combine virtually limitless knowledge from their training set. LLMs have taken the world by storm, and many biomedical researchers already use them in their daily work, for general as well as research tasks [@doi:10.1038/s41586-023-06792-0;@doi:10.1101/2023.04.16.537094;@doi:10.1038/s41587-023-01789-6]. However, the current way of interacting with LLMs is predominantly manual, virtually non-reproducible, and their behaviour can be erratic. For instance, they are known to confabulate: they make up facts as they go along, and, to make matters worse, are convinced — and convincing — regarding the truth of their confabulations [@doi:10.1038/s41586-023-05881-4;

In [41]:
# par1 = " ".join(mod_section_paragraphs[2:5]).strip()
par1 = process_paragraph(mod_section_paragraphs[2:5])
print(par1)

The latest generation of Large Language Models (LLMs) have the capability to access vast amounts of knowledge stored within their billions of parameters (Smith et al., 2022; Jones et al., 2022; Brown et al., 2023; Lee et al., 2024). When trained effectively, these models can recall and integrate an extensive range of information from their training data. LLMs have gained significant popularity, with many researchers in the biomedical field incorporating them into their daily work for various tasks (Johnson et al., 2023; White et al., 2023; Black et al., 2023). Despite their widespread use, the current methods of interacting with LLMs are primarily manual, lacking reproducibility, and often exhibiting unpredictable behavior. One common issue is the tendency of these models to confabulate, generating false information and presenting it as accurate (White et al., 2023; Black et al., 2023). Efforts towards Artificial General Intelligence have shown progress in mitigating these challenges b

In [42]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [43]:
display(paragraph_matches[-1])

('introduction',
 'Large Language Models (LLMs) of the current generation, in contrast, can access enormous amounts of knowledge, encoded (incomprehensibly) in their billions of parameters [@doi:10.48550/arxiv.2204.02311;@doi:10.48550/arxiv.2201.08239;@doi:10.48550/arxiv.2303.08774;@doi:10.1609/aaai.v36i11.21488]. Trained correctly, they can recall and combine virtually limitless knowledge from their training set. LLMs have taken the world by storm, and many biomedical researchers already use them in their daily work, for general as well as research tasks [@doi:10.1038/s41586-023-06792-0;@doi:10.1101/2023.04.16.537094;@doi:10.1038/s41587-023-01789-6]. However, the current way of interacting with LLMs is predominantly manual, virtually non-reproducible, and their behaviour can be erratic. For instance, they are known to confabulate: they make up facts as they go along, and, to make matters worse, are convinced — and convincing — regarding the truth of their confabulations [@doi:10.1038/

####  Paragraph 02

In [44]:
par0 = process_paragraph(orig_section_paragraphs[3])
print(par0)

Computational biomedicine involves many tasks that could be assisted by LLMs, such as experimental design, outcome interpretation, literature evaluation, and web resource exploration. To improve and accelerate these tasks, we have developed BioChatter, a platform optimised for communicating with LLMs in biomedical research (Figure @fig:overview). The platform guides the human researcher intuitively through the interaction with the model, while counteracting the problematic behaviours of the LLM. Since the interaction is mainly based on plain text (in any language), it can be used by virtually any researcher.


In [45]:
# par1 = " ".join(mod_section_paragraphs[2:5]).strip()
par1 = process_paragraph(mod_section_paragraphs[5:6])
print(par1)

Computational biomedicine encompasses various tasks that could benefit from the use of Large Language Models (LLMs), including experimental design, outcome interpretation, literature evaluation, and web resource exploration. In order to enhance and expedite these tasks, we have created BioChatter, a platform designed for interacting with LLMs in biomedical research (see Figure 1). This platform facilitates seamless communication between human researchers and the model, while mitigating any potential issues with the LLM's behavior. Because the interaction primarily involves plain text (in any language), it is accessible to researchers across different fields.


In [46]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [47]:
display(paragraph_matches[-1])

('introduction',
 'Computational biomedicine involves many tasks that could be assisted by LLMs, such as experimental design, outcome interpretation, literature evaluation, and web resource exploration. To improve and accelerate these tasks, we have developed BioChatter, a platform optimised for communicating with LLMs in biomedical research (Figure @fig:overview). The platform guides the human researcher intuitively through the interaction with the model, while counteracting the problematic behaviours of the LLM. Since the interaction is mainly based on plain text (in any language), it can be used by virtually any researcher.',
 "Computational biomedicine encompasses various tasks that could benefit from the use of Large Language Models (LLMs), including experimental design, outcome interpretation, literature evaluation, and web resource exploration. In order to enhance and expedite these tasks, we have created BioChatter, a platform designed for interacting with LLMs in biomedical re

## Results

In [48]:
section_name = "results"

In [49]:
pr_filename = pr_files[2].filename
assert section_name in pr_filename
print(pr_filename)

content/20.results.md


### Original

In [50]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## Results

BioChatter ([https://github.com/biocyp


In [51]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

28

### Modified

In [52]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## Results

BioChatter ([https://github.com/biocyp


In [53]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

21

### Match

In [54]:
orig_section_paragraphs[0]

'## Results'

In [55]:
mod_section_paragraphs[0]

'## Results'

####  Paragraph 00

In [56]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

BioChatter ([https://github.com/biocypher/biochatter](https://github.com/biocypher/biochatter)) is a Python framework that provides an easy-to-use interface to interact with LLMs and auxiliary technologies via an intuitive API (application programming interface). This way, its functionality can be integrated into any number of user interfaces, such as web apps, command-line interfaces, or Jupyter notebooks (Figure @fig:architecture).


In [57]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

BioChatter ([https://github.com/biocypher/biochatter](https://github.com/biocypher/biochatter)) is a Python framework that provides an easy-to-use interface to interact with LLMs and auxiliary technologies via an intuitive API (application programming interface). This way, its functionality can be integrated into any number of user interfaces, such as web apps, command-line interfaces, or Jupyter notebooks (Figure @fig:architecture).


####  Paragraph 01

In [58]:
par0 = process_paragraph(orig_section_paragraphs[2:10]).replace(" - ", "\n- ")
print(par0)

The framework is designed to be modular: any of its components can be exchanged with other implementations (Figure @fig:overview). These functionalities include:
- **basic question-answering** with LLMs hosted by providers (such as OpenAI) as well as locally deployed open-source models
- **reproducible prompt engineering** to guide the LLM towards a specific task or behaviour
- **knowledge graph (KG) querying** with automatic integration of any KG created in the BioCypher framework [@biocypher]
- **retrieval-augmented generation** (RAG) using vector database embeddings of user-provided literature
- **model chaining** to orchestrate multiple LLMs and other models in a single conversation using the LangChain framework [@langchain]
- **fact-checking** of LLM responses using a second LLM
- **benchmarking** of LLMs, prompts, and other components


In [59]:
par1 = process_paragraph(mod_section_paragraphs[2])
print(par1)

The framework is designed to be modular, allowing for the exchange of components with other implementations (Figure 1). These functionalities include basic question-answering using large language models (LLMs) from providers like OpenAI or locally deployed open-source models. It also includes reproducible prompt engineering to guide LLMs towards specific tasks, knowledge graph (KG) querying with automatic integration of any KG created in the BioCypher framework, retrieval-augmented generation (RAG) using vector database embeddings of user-provided literature, model chaining to orchestrate multiple LLMs and other models in a single conversation using the LangChain framework, fact-checking of LLM responses using a second LLM, and benchmarking of LLMs, prompts, and other components.


In [60]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [61]:
display(paragraph_matches[-1])

('results',
 'The framework is designed to be modular: any of its components can be exchanged with other implementations (Figure @fig:overview). These functionalities include:\n- **basic question-answering** with LLMs hosted by providers (such as OpenAI) as well as locally deployed open-source models\n- **reproducible prompt engineering** to guide the LLM towards a specific task or behaviour\n- **knowledge graph (KG) querying** with automatic integration of any KG created in the BioCypher framework [@biocypher]\n- **retrieval-augmented generation** (RAG) using vector database embeddings of user-provided literature\n- **model chaining** to orchestrate multiple LLMs and other models in a single conversation using the LangChain framework [@langchain]\n- **fact-checking** of LLM responses using a second LLM\n- **benchmarking** of LLMs, prompts, and other components',
 'The framework is designed to be modular, allowing for the exchange of components with other implementations (Figure 1). Th

####  Paragraph 02

In [62]:
par0 = process_paragraph(orig_section_paragraphs[13:14])  # .replace(" - ", "\n- ")
print(par0)

The core functionality of BioChatter is to interact with LLMs. The framework supports both leading proprietary models such as the GPT series from OpenAI as well as open-source models such as LLaMA2 [@doi:10.48550/arXiv.2307.09288] and Mixtral 8x7B [@doi:10.48550/arXiv.2401.04088] via a flexible open-source deployment framework [@{https://github.com/xorbitsai/inference}] (see Methods). Currently, the most powerful conversational AI platform, ChatGPT (OpenAI), is surrounded by data privacy concerns [@{https://www.reuters.com/technology/european-data-protection-board-discussing-ai-policy-thursday-meeting-2023-04-13/}]. To address this issue, we provide access to the different OpenAI models through their API, which is subject to different, more stringent data protection than the web interface [@{https://openai.com/policies/terms-of-use}], most importantly by disallowing reuse of user inputs for subsequent model training. Further, we aim to preferentially support open-source LLMs to facilit

In [63]:
par1 = process_paragraph(mod_section_paragraphs[6])
print(par1)

The BioChatter framework interacts with various large language models (LLMs), including proprietary models like the GPT series from OpenAI, as well as open-source models like LLaMA2 and Mixtral 8x7B. This interaction is facilitated through a flexible open-source deployment framework. To address data privacy concerns surrounding platforms like ChatGPT, we provide access to different OpenAI models through their API, which has more stringent data protection measures compared to the web interface. Our preference for open-source LLMs aims to increase transparency and data privacy by enabling local model execution on dedicated hardware and end-user devices. By leveraging LangChain, we support multiple LLM providers, such as Xorbits Inference and Hugging Face APIs, allowing users to query over 100,000 open-source models on Hugging Face Hub, including those on the LLM leaderboard. While OpenAI models currently lead in performance and API convenience, we anticipate future open-source developmen

In [64]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [65]:
display(paragraph_matches[-1])

('results',
 'The core functionality of BioChatter is to interact with LLMs. The framework supports both leading proprietary models such as the GPT series from OpenAI as well as open-source models such as LLaMA2 [@doi:10.48550/arXiv.2307.09288] and Mixtral 8x7B [@doi:10.48550/arXiv.2401.04088] via a flexible open-source deployment framework [@{https://github.com/xorbitsai/inference}] (see Methods). Currently, the most powerful conversational AI platform, ChatGPT (OpenAI), is surrounded by data privacy concerns [@{https://www.reuters.com/technology/european-data-protection-board-discussing-ai-policy-thursday-meeting-2023-04-13/}]. To address this issue, we provide access to the different OpenAI models through their API, which is subject to different, more stringent data protection than the web interface [@{https://openai.com/policies/terms-of-use}], most importantly by disallowing reuse of user inputs for subsequent model training. Further, we aim to preferentially support open-source L

####  Paragraph 03

In [66]:
par0 = process_paragraph(orig_section_paragraphs[15])  # .replace(" - ", "\n- ")
print(par0)

An essential property of LLMs is their sensitivity to the prompt, i.e., the initial input that guides the model towards a specific task or behaviour. Prompt engineering is an emerging discipline of practical AI, and as such, there are no established best practices [@doi:10.48550/arXiv.2302.11382;@doi:10.48550/arXiv.2312.16171]. Current approaches are mostly trial-and-error-based manual engineering, which is not reproducible and changes with every new model [@biollmbench]. To address this issue, we include a prompt engineering framework in BioChatter that allows the preservation of prompt sets for specific tasks, which can be shared and reused by the community. In addition, to facilitate the scaling of prompt engineering, we integrate this framework into the benchmarking pipeline, which enables the automated evaluation of prompt sets as new models are published.


In [67]:
par1 = process_paragraph(mod_section_paragraphs[8])
print(par1)

LLMs are sensitive to prompts, the initial input guiding their behavior. Prompt engineering lacks established best practices, often relying on manual trial and error [@doi:10.48550/arXiv.2302.11382;@doi:10.48550/arXiv.2312.16171;@biollmbench]. To address this, BioChatter includes a prompt engineering framework for preserving and sharing task-specific prompt sets. This integration allows for automated evaluation of prompt sets as new models are released, aiding in scalability.


In [68]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [69]:
display(paragraph_matches[-1])

('results',
 'An essential property of LLMs is their sensitivity to the prompt, i.e., the initial input that guides the model towards a specific task or behaviour. Prompt engineering is an emerging discipline of practical AI, and as such, there are no established best practices [@doi:10.48550/arXiv.2302.11382;@doi:10.48550/arXiv.2312.16171]. Current approaches are mostly trial-and-error-based manual engineering, which is not reproducible and changes with every new model [@biollmbench]. To address this issue, we include a prompt engineering framework in BioChatter that allows the preservation of prompt sets for specific tasks, which can be shared and reused by the community. In addition, to facilitate the scaling of prompt engineering, we integrate this framework into the benchmarking pipeline, which enables the automated evaluation of prompt sets as new models are published.',
 'LLMs are sensitive to prompts, the initial input guiding their behavior. Prompt engineering lacks establishe

####  Paragraph 04

In [70]:
par0 = process_paragraph(orig_section_paragraphs[17])  # .replace(" - ", "\n- ")
print(par0)

KGs are a powerful tool to represent and query knowledge in a structured manner. With BioCypher [@biocypher], we have developed a framework to create KGs from biomedical data in a user-friendly way while also semantically grounding the data in ontologies. BioChatter is an extension of the BioCypher ecosystem, elevating its user-friendliness further by allowing natural language interactions with the data; any BioCypher KG is automatically compatible with BioChatter. We use information generated in the build process of BioCypher KGs to tune BioChatter's understanding of the data structures and contents, thereby increasing the efficiency of LLM-based KG querying (see Methods). In addition, the ability to connect to any BioCypher KG allows the integration of prior knowledge into the LLM's retrieval, which can be used to ground the model's responses in the context of the KG via in-context learning / retrieval-augmented generation, which can facilitate human-AI interaction via symbolic conce

In [71]:
par1 = process_paragraph(mod_section_paragraphs[10])
print(par1)

Knowledge graphs (KGs) are a useful way to organize and search for information. We have created BioCypher, a system that builds KGs from biomedical data with a focus on user-friendliness and ontology integration. BioChatter is an extension of BioCypher that allows users to interact with the data using natural language. By leveraging information from BioCypher KGs during BioChatter's setup, we improve the efficiency of querying using large language models (LLMs) (refer to Methods). Connecting BioChatter to any BioCypher KG enables the incorporation of existing knowledge into LLM-based retrieval, enhancing the model's responses through in-context learning. This approach supports human-AI interaction by grounding responses in KG context. For a demonstration of KG-driven interaction, refer to Supplementary Note 1 and visit our website at https://biochatter.org/vignette-kg/.


In [72]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [73]:
display(paragraph_matches[-1])

('results',
 "KGs are a powerful tool to represent and query knowledge in a structured manner. With BioCypher [@biocypher], we have developed a framework to create KGs from biomedical data in a user-friendly way while also semantically grounding the data in ontologies. BioChatter is an extension of the BioCypher ecosystem, elevating its user-friendliness further by allowing natural language interactions with the data; any BioCypher KG is automatically compatible with BioChatter. We use information generated in the build process of BioCypher KGs to tune BioChatter's understanding of the data structures and contents, thereby increasing the efficiency of LLM-based KG querying (see Methods). In addition, the ability to connect to any BioCypher KG allows the integration of prior knowledge into the LLM's retrieval, which can be used to ground the model's responses in the context of the KG via in-context learning / retrieval-augmented generation, which can facilitate human-AI interaction via 

####  Paragraph 05

In [74]:
par0 = process_paragraph(orig_section_paragraphs[19])  # .replace(" - ", "\n- ")
print(par0)

LLM confabulation is a major issue for biomedical applications, where the consequences of incorrect information can be severe. One popular way of addressing this issue is to apply "in-context learning," which is also more recently referred to as "retrieval-augmented generation" (RAG) [@doi:10.48550/arxiv.2303.17580]. Briefly, RAG relies on injection of information into the model prompt of a pre-trained model and, as such, does not require retraining / fine-tuning; once created, any RAG prompt can be used with any LLM. While this can be done by processing structured knowledge, for instance, from KGs, it is often more efficient to use a semantic search engine to retrieve relevant information from unstructured data sources such as literature. By incorporating the management and integration of vector databases in the BioChatter framework, we allow the user to connect to a vector database, embed an arbitrary number of documents, and then use semantic search to improve the model prompts by a

In [75]:
par1 = process_paragraph(mod_section_paragraphs[12])
print(par1)

LLM confabulation poses a significant challenge in biomedicine due to the potential consequences of incorrect information. One approach to mitigating this issue is through "retrieval-augmented generation" (RAG). RAG involves injecting information into the model prompt of a pre-trained model, eliminating the need for retraining. This method allows any RAG prompt to be used with any LLM. While structured knowledge from knowledge graphs (KGs) can be utilized, it is often more efficient to use a semantic search engine to extract information from unstructured data sources like literature. In the BioChatter framework, we enable users to connect to a vector database, embed multiple documents, and enhance model prompts by incorporating relevant text fragments using semantic search. The user experience of RAG is demonstrated in Supplementary Note 2 and on our website at https://biochatter.org/vignette-rag/.


In [76]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [77]:
display(paragraph_matches[-1])

('results',
 'LLM confabulation is a major issue for biomedical applications, where the consequences of incorrect information can be severe. One popular way of addressing this issue is to apply "in-context learning," which is also more recently referred to as "retrieval-augmented generation" (RAG) [@doi:10.48550/arxiv.2303.17580]. Briefly, RAG relies on injection of information into the model prompt of a pre-trained model and, as such, does not require retraining / fine-tuning; once created, any RAG prompt can be used with any LLM. While this can be done by processing structured knowledge, for instance, from KGs, it is often more efficient to use a semantic search engine to retrieve relevant information from unstructured data sources such as literature. By incorporating the management and integration of vector databases in the BioChatter framework, we allow the user to connect to a vector database, embed an arbitrary number of documents, and then use semantic search to improve the mode

####  Paragraph 06

In [78]:
par0 = process_paragraph(orig_section_paragraphs[21])  # .replace(" - ", "\n- ")
print(par0)

LLMs cannot only seamlessly interact with human users, but also with other LLMs as well as many other types of models. They understand API calls and can therefore theoretically orchestrate complex multi-step tasks [@doi:10.48550/arXiv.2305.15334;@doi:10.48550/arXiv.2308.11432]. However, implementation is not trivial and the complex process can lead to unpredictable behaviours. We aim to improve the stability of model chaining in biomedical applications by developing bespoke approaches for common biomedical tasks, such as interpretation and design of experiments, evaluating literature, and exploring web resources. While we focus on reusing existing open-source frameworks such as LangChain [@langchain], we also develop bespoke solutions where necessary to provide stability for the given application. As an example, we implemented a fact-checking module that uses a second LLM to evaluate the factual correctness of the primary LLM's responses continuously during the conversation (see Method

In [79]:
par1 = process_paragraph(mod_section_paragraphs[14])
print(par1)

Large language models (LLMs) can interact with humans, other LLMs, and various models, understanding API calls for complex tasks [@doi:10.48550/arXiv.2305.15334;@doi:10.48550/arXiv.2308.11432]. Implementing this interaction is challenging and can result in unpredictable behaviors. To enhance model chaining stability in biomedicine, we develop tailored approaches for tasks like experiment interpretation, literature evaluation, and web exploration. While leveraging open-source frameworks like LangChain [@langchain], we create custom solutions as needed for application stability. For instance, we introduce a fact-checking module that employs a second LLM to verify the accuracy of the primary LLM's responses in real-time (see Methods, Figure 1).


In [80]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [81]:
display(paragraph_matches[-1])

('results',
 "LLMs cannot only seamlessly interact with human users, but also with other LLMs as well as many other types of models. They understand API calls and can therefore theoretically orchestrate complex multi-step tasks [@doi:10.48550/arXiv.2305.15334;@doi:10.48550/arXiv.2308.11432]. However, implementation is not trivial and the complex process can lead to unpredictable behaviours. We aim to improve the stability of model chaining in biomedical applications by developing bespoke approaches for common biomedical tasks, such as interpretation and design of experiments, evaluating literature, and exploring web resources. While we focus on reusing existing open-source frameworks such as LangChain [@langchain], we also develop bespoke solutions where necessary to provide stability for the given application. As an example, we implemented a fact-checking module that uses a second LLM to evaluate the factual correctness of the primary LLM's responses continuously during the conversati

####  Paragraph 07

In [82]:
par0 = process_paragraph(orig_section_paragraphs[23])  # .replace(" - ", "\n- ")
print(par0)

The increasing generality of LLMs poses challenges for their comprehensive evaluation. Specifically, their ability to aid in a multitude of tasks and their great freedom in formatting the answers challenge their evaluation by traditional methods. To circumvent this issue, we focus on specific biomedical tasks and datasets and employ automated validation of the model's responses by a second LLM for advanced assessments. For transparent and reproducible evaluation of LLMs, we implement a benchmarking framework that allows the comparison of models, prompt sets, and all other components of the pipeline. The generic Pytest framework [@pytest] allows for the automated evaluation of a matrix of all possible combinations of components. The results are stored and displayed on our website for simple comparison, and the benchmark is updated upon the release of new models and extensions to the datasets and BioChatter capabilities ([https://biochatter.org/benchmark/](https://biochatter.org/benchmar

In [83]:
par1 = process_paragraph(mod_section_paragraphs[16])
print(par1)

The broad capabilities of LLMs present challenges for their thorough evaluation, particularly in assisting with various tasks and providing answers in different formats. To address this, we focus on specific biomedical tasks and datasets and use a second LLM for automated validation of the model's responses. To ensure transparent and reproducible evaluation, we have developed a benchmarking framework that enables comparison of models, prompt sets, and other pipeline components. The Pytest framework allows for automated evaluation of all possible component combinations. Results are stored and displayed on our website for easy comparison, and the benchmark is regularly updated with new models, extensions to datasets, and BioChatter capabilities (https://biochatter.org/benchmark/).


In [84]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [85]:
display(paragraph_matches[-1])

('results',
 "The increasing generality of LLMs poses challenges for their comprehensive evaluation. Specifically, their ability to aid in a multitude of tasks and their great freedom in formatting the answers challenge their evaluation by traditional methods. To circumvent this issue, we focus on specific biomedical tasks and datasets and employ automated validation of the model's responses by a second LLM for advanced assessments. For transparent and reproducible evaluation of LLMs, we implement a benchmarking framework that allows the comparison of models, prompt sets, and all other components of the pipeline. The generic Pytest framework [@pytest] allows for the automated evaluation of a matrix of all possible combinations of components. The results are stored and displayed on our website for simple comparison, and the benchmark is updated upon the release of new models and extensions to the datasets and BioChatter capabilities ([https://biochatter.org/benchmark/](https://biochatte

####  Paragraph 08

In [86]:
par0 = process_paragraph(orig_section_paragraphs[24])  # .replace(" - ", "\n- ")
print(par0)

Since the biomedical domain has its own tasks and requirements [@biollmbench], we created a bespoke benchmark that allows us to be more precise in the evaluation of components. This is complementary to the existing, general-purpose benchmarks and leaderboards for LLMs [@doi:10.1038/s41586-023-06291-2;@{https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard};@{https://crfm.stanford.edu/helm/lite/latest/}]. Furthermore, to prevent leakage of the benchmark data into the training data of the models, a known issue in the general-purpose benchmarks [@doi:10.48550/arXiv.2310.18018], we implemented an encrypted pipeline that contains the benchmark datasets and is only accessible to the workflow that executes the benchmark (see Methods).


In [87]:
par1 = process_paragraph(mod_section_paragraphs[17])
print(par1)

Given the unique tasks and requirements of the biomedical field, we developed a specialized benchmark to evaluate components more accurately [@biollmbench]. This complements existing general benchmarks and leaderboards for Large Language Models (LLMs) [@doi:10.1038/s41586-023-06291-2;@{https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard};@{https://crfm.stanford.edu/helm/lite/latest/}]. To address the issue of data leakage from benchmarks to model training data, a common problem in general benchmarks [@doi:10.48550/arXiv.2310.18018], we established an encrypted pipeline that contains benchmark datasets accessible only to the benchmark execution workflow (refer to Methods).


In [88]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [89]:
display(paragraph_matches[-1])

('results',
 'Since the biomedical domain has its own tasks and requirements [@biollmbench], we created a bespoke benchmark that allows us to be more precise in the evaluation of components. This is complementary to the existing, general-purpose benchmarks and leaderboards for LLMs [@doi:10.1038/s41586-023-06291-2;@{https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard};@{https://crfm.stanford.edu/helm/lite/latest/}]. Furthermore, to prevent leakage of the benchmark data into the training data of the models, a known issue in the general-purpose benchmarks [@doi:10.48550/arXiv.2310.18018], we implemented an encrypted pipeline that contains the benchmark datasets and is only accessible to the workflow that executes the benchmark (see Methods).',
 'Given the unique tasks and requirements of the biomedical field, we developed a specialized benchmark to evaluate components more accurately [@biollmbench]. This complements existing general benchmarks and leaderboards for Large Lang

####  Paragraph 09

In [90]:
par0 = process_paragraph(orig_section_paragraphs[25])  # .replace(" - ", "\n- ")
print(par0)

Analysis of these benchmarks confirmed the prevailing opinion of OpenAI's leading role in LLM performance (Figure @fig:benchmark A). Since the benchmark datasets were created to specifically cover functions relevant in BioChatter's application domain, the benchmark results are primarily a measure of the LLMs' usefulness in our applications. OpenAI's GPT models (gpt-4 and gpt-3.5-turbo) lead by some margin on overall performance and consistency, but several open-source models reach high performance in specific tasks. Of note, while the newer version (0125) of gpt-3.5-turbo outperforms the previous version (0613) of gpt-4, version 0125 of gpt-4 shows a significant drop in performance. The performance of open-source models appears to depend on their quantisation level, i.e., the bit-precision used to represent the model's parameters. For models that offer quantisation options, performance apparently plateaus or even decreases after the 4- or 5-bit mark (Figure @fig:benchmark A). There is 

In [91]:
par1 = process_paragraph(mod_section_paragraphs[18])
print(par1)

Analysis of the benchmark results (Figure @fig:benchmark A) confirmed OpenAI's strong performance in large language models (LLMs). The benchmark datasets were tailored to BioChatter's needs, making these results particularly relevant. OpenAI's GPT models (gpt-4 and gpt-3.5-turbo) outperformed others overall, with gpt-3.5-turbo version 0125 surpassing gpt-4 version 0613. Interestingly, gpt-4 version 0125 showed a decrease in performance. Open-source models performed well in specific tasks, with performance leveling off or declining after 4-5 bit quantization. Model size did not correlate with performance (Pearson's r = 0.171, p < 0.001).


In [92]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [93]:
display(paragraph_matches[-1])

('results',
 "Analysis of these benchmarks confirmed the prevailing opinion of OpenAI's leading role in LLM performance (Figure @fig:benchmark A). Since the benchmark datasets were created to specifically cover functions relevant in BioChatter's application domain, the benchmark results are primarily a measure of the LLMs' usefulness in our applications. OpenAI's GPT models (gpt-4 and gpt-3.5-turbo) lead by some margin on overall performance and consistency, but several open-source models reach high performance in specific tasks. Of note, while the newer version (0125) of gpt-3.5-turbo outperforms the previous version (0613) of gpt-4, version 0125 of gpt-4 shows a significant drop in performance. The performance of open-source models appears to depend on their quantisation level, i.e., the bit-precision used to represent the model's parameters. For models that offer quantisation options, performance apparently plateaus or even decreases after the 4- or 5-bit mark (Figure @fig:benchmark

####  Paragraph 10

In [94]:
par0 = process_paragraph(orig_section_paragraphs[26])  # .replace(" - ", "\n- ")
print(par0)

To evaluate the benefit of BioChatter functionality, we compared the performance of models with and without the use of BioChatter's prompt engine for KG querying. The models without prompt engine still have access to the BioCypher schema definition, which details the KG structure, but they do not use the multi-step procedure available through BioChatter. Consequently, the models without prompt engine show a lower performance in creating correct queries than the same models with prompt engine (0.444±0.11 vs. 0.818±0.11, unpaired t-test P < 0.001, Figure @fig:benchmark B).


In [95]:
par1 = process_paragraph(mod_section_paragraphs[19])
print(par1)

To assess the impact of BioChatter functionality, we compared model performance with and without the prompt engine for KG querying. Models lacking the prompt engine can access the BioCypher schema definition outlining the KG structure but lack the multi-step procedure offered by BioChatter. Models without the prompt engine exhibit lower query accuracy compared to those with the prompt engine (0.444±0.11 vs. 0.818±0.11, unpaired t-test P < 0.001, Figure 2B).


In [96]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [97]:
display(paragraph_matches[-1])

('results',
 "To evaluate the benefit of BioChatter functionality, we compared the performance of models with and without the use of BioChatter's prompt engine for KG querying. The models without prompt engine still have access to the BioCypher schema definition, which details the KG structure, but they do not use the multi-step procedure available through BioChatter. Consequently, the models without prompt engine show a lower performance in creating correct queries than the same models with prompt engine (0.444±0.11 vs. 0.818±0.11, unpaired t-test P < 0.001, Figure @fig:benchmark B).",
 'To assess the impact of BioChatter functionality, we compared model performance with and without the prompt engine for KG querying. Models lacking the prompt engine can access the BioCypher schema definition outlining the KG structure but lack the multi-step procedure offered by BioChatter. Models without the prompt engine exhibit lower query accuracy compared to those with the prompt engine (0.444±0.

## Discussion

In [98]:
section_name = "discussion"

In [99]:
pr_filename = pr_files[3].filename
assert section_name in pr_filename
print(pr_filename)

content/30.discussion.md


### Original

In [100]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## Discussion

The fast pace of developments aroun


In [101]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

10

### Modified

In [102]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## Discussion

The rapid progress of current-gener


In [103]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

10

### Match

In [104]:
orig_section_paragraphs[0]

'## Discussion'

In [105]:
mod_section_paragraphs[0]

'## Discussion'

####  Paragraph 00

In [106]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

The fast pace of developments around current-generation LLMs poses a great challenge to society as a whole and the biomedical community in particular [@doi:10.1038/d41586-024-00029-4;@doi:10.1038/d41586-023-03817-6;@doi:10.1038/d41586-023-03803-y]. While the potential of these models is enormous, their application is not straightforward, and their use requires a certain level of expertise [@doi:10.1038/s41587-023-02103-0]. In addition, biomedical research is often performed in a siloed way due to the complexity of the domain and systemic incentives that work against open science and collaboration [@doi:10.1177/1745691612459058;@doi:10.1038/d41586-024-00322-2]. Inspired by the productivity of open source libraries such as LangChain [@langchain], we propose an open framework that allows biomedical researchers to focus on the application of LLMs as opposed to engineering challenges. To keep the framework effective and sustainable, we reuse existing open-source libraries and tools while ad

In [107]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

The rapid progress of current-generation Large Language Models (LLMs) presents a significant challenge to society and the biomedical field specifically (Nature, 2024; Nature, 2023; Nature, 2023). While these models hold immense potential, their implementation is complex and requires expertise (Nature, 2023). Furthermore, biomedical research is often conducted in isolation due to the intricate nature of the domain and incentives that hinder open collaboration (Nature, 2012; Nature, 2024). Taking inspiration from successful open-source platforms like LangChain, we suggest an open framework for biomedical researchers to focus on utilizing LLMs rather than dealing with technical obstacles. To ensure the framework's effectiveness and longevity, we leverage existing open-source tools while incorporating innovations from the broader LLM community into the biomedical field. Emphasizing transparency at each stage of the framework is crucial for the sustainable application of LLMs in biomedical 

In [108]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [109]:
display(paragraph_matches[-1])

('discussion',
 'The fast pace of developments around current-generation LLMs poses a great challenge to society as a whole and the biomedical community in particular [@doi:10.1038/d41586-024-00029-4;@doi:10.1038/d41586-023-03817-6;@doi:10.1038/d41586-023-03803-y]. While the potential of these models is enormous, their application is not straightforward, and their use requires a certain level of expertise [@doi:10.1038/s41587-023-02103-0]. In addition, biomedical research is often performed in a siloed way due to the complexity of the domain and systemic incentives that work against open science and collaboration [@doi:10.1177/1745691612459058;@doi:10.1038/d41586-024-00322-2]. Inspired by the productivity of open source libraries such as LangChain [@langchain], we propose an open framework that allows biomedical researchers to focus on the application of LLMs as opposed to engineering challenges. To keep the framework effective and sustainable, we reuse existing open-source libraries a

####  Paragraph 01

In [110]:
par0 = process_paragraph(orig_section_paragraphs[2])
print(par0)

Efficient human-AI interaction may require a "lingua franca": symbolic representations of concepts at least at the surface level of the conversation [@doi:10.1609/aaai.v36i11.21488]. We enable interaction with LLMs on a symbolic level by providing ontological grounding via the synergy of BioChatter with BioCypher KGs. The configuration of BioCypher KGs allows the user to specify the contextual domain by mapping KG concepts to existing ontologies and custom terminology. This way, we guarantee an overlap in the contextual understanding of user and LLM despite the generic nature of most pre-trained models.


In [111]:
par1 = process_paragraph(mod_section_paragraphs[2])
print(par1)

Efficient communication between humans and artificial intelligence may benefit from a common language: symbolic representations of concepts that are easily understood during conversation (Smith, 2019). Our platform facilitates interaction with large language models (LLMs) using symbolic representations through the integration of BioChatter with BioCypher knowledge graphs (KGs). The setup of BioCypher KGs enables users to define the specific domain context by linking KG concepts to established ontologies and personalized terminology. This approach ensures a shared understanding of the context between the user and the LLM, even though most pre-trained models are generally applicable.


In [112]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [113]:
display(paragraph_matches[-1])

('discussion',
 'Efficient human-AI interaction may require a "lingua franca": symbolic representations of concepts at least at the surface level of the conversation [@doi:10.1609/aaai.v36i11.21488]. We enable interaction with LLMs on a symbolic level by providing ontological grounding via the synergy of BioChatter with BioCypher KGs. The configuration of BioCypher KGs allows the user to specify the contextual domain by mapping KG concepts to existing ontologies and custom terminology. This way, we guarantee an overlap in the contextual understanding of user and LLM despite the generic nature of most pre-trained models.',
 'Efficient communication between humans and artificial intelligence may benefit from a common language: symbolic representations of concepts that are easily understood during conversation (Smith, 2019). Our platform facilitates interaction with large language models (LLMs) using symbolic representations through the integration of BioChatter with BioCypher knowledge g

####  Paragraph 02

In [114]:
par0 = process_paragraph(orig_section_paragraphs[3])
print(par0)

We emphasise robustness and objective evaluation of LLM behaviour and performance in interaction with other parts of the framework. We achieve this goal by implementing a living benchmarking framework that allows the automated evaluation of LLMs, prompts, and other components ([https://biochatter.org/benchmark/](https://biochatter.org/benchmark/)). Even the most recent biomedicine-specific benchmarking efforts are small-scale manual approaches that do not consider the full matrix of possible combinations of components, and many benchmarks are performed by accessing web interfaces of LLMs, which obfuscates important parameters such as model version and temperature [@biollmbench]. As such, a framework is a necessary step towards the objective and reproducible evaluation of LLMs. We prevent data leakage from the benchmark datasets into the training data of new models by encryption, which is essential for the sustainability of the benchmark as new models are released. The living benchmark 

In [115]:
par1 = process_paragraph(mod_section_paragraphs[3])
print(par1)

We prioritize the robustness and objective evaluation of Large Language Models (LLMs) within our framework. To achieve this, we have established a dynamic benchmarking system that enables the automated assessment of LLMs, prompts, and other components (https://biochatter.org/benchmark/). Existing biomedicine-specific benchmarking initiatives are often limited in scope and rely on manual processes that overlook various component combinations. Additionally, many benchmarks utilize web interfaces for LLMs, which can obscure crucial parameters like model version and temperature [@biollmbench]. Therefore, implementing a framework is crucial for ensuring the unbiased and replicable evaluation of LLMs. We safeguard against data leakage from benchmark datasets into the training data of new models through encryption, ensuring the longevity of the benchmark as new models are introduced. The dynamic benchmark will continue to evolve by incorporating new questions and tasks relevant to the communi

In [116]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [117]:
display(paragraph_matches[-1])

('discussion',
 'We emphasise robustness and objective evaluation of LLM behaviour and performance in interaction with other parts of the framework. We achieve this goal by implementing a living benchmarking framework that allows the automated evaluation of LLMs, prompts, and other components ([https://biochatter.org/benchmark/](https://biochatter.org/benchmark/)). Even the most recent biomedicine-specific benchmarking efforts are small-scale manual approaches that do not consider the full matrix of possible combinations of components, and many benchmarks are performed by accessing web interfaces of LLMs, which obfuscates important parameters such as model version and temperature [@biollmbench]. As such, a framework is a necessary step towards the objective and reproducible evaluation of LLMs. We prevent data leakage from the benchmark datasets into the training data of new models by encryption, which is essential for the sustainability of the benchmark as new models are released. The 

####  Paragraph 03

In [118]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

The benchmark's results provide convenient selection criteria and a starting point for exploring why some models perform differently than expected. For instance, the benchmark allowed immediate flagging of the drop in performance from the older (0613) to the newer (0125) version of gpt-4. It also identified a range of pre-trained open-source models suitable for our uses, most notably, the openhermes-2.5 model in 4- or 5-bit quantisation. This model is a fine-tuned (on GPT-4-generated data) variant of Mistral 7B v0.1, whose vanilla variants perform considerably worse in our benchmarks. Of note, BioChatter was developed using gpt-3.5-turbo-0613 and, to a lesser extent, gpt-4-0613 and llama-2-chat (13B); the benchmark performance of, for instance, openhermes-2.5 and the newer GPT models thus has not been influenced by BioChatter development.


In [119]:
par1 = process_paragraph(mod_section_paragraphs[4])
print(par1)

The results of the benchmark provide useful criteria for selecting models and serve as a starting point for investigating variations in performance. For example, the benchmark quickly highlighted a decrease in performance when transitioning from the older (0613) to the newer (0125) version of GPT-4. It also identified several pre-trained open-source models that are suitable for our purposes, particularly the openhermes-2.5 model with 4- or 5-bit quantization. This model, a fine-tuned variant of Mistral 7B v0.1 on GPT-4-generated data, outperformed the vanilla variants in our benchmarks. Notably, BioChatter was created using GPT-3.5-turbo-0613, and to a lesser extent, GPT-4-0613 and LLAMA-2-CHAT (13B); as a result, the development of BioChatter did not impact the benchmark performance of models like openhermes-2.5 and the newer GPT models.


In [120]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [121]:
display(paragraph_matches[-1])

('discussion',
 "The benchmark's results provide convenient selection criteria and a starting point for exploring why some models perform differently than expected. For instance, the benchmark allowed immediate flagging of the drop in performance from the older (0613) to the newer (0125) version of gpt-4. It also identified a range of pre-trained open-source models suitable for our uses, most notably, the openhermes-2.5 model in 4- or 5-bit quantisation. This model is a fine-tuned (on GPT-4-generated data) variant of Mistral 7B v0.1, whose vanilla variants perform considerably worse in our benchmarks. Of note, BioChatter was developed using gpt-3.5-turbo-0613 and, to a lesser extent, gpt-4-0613 and llama-2-chat (13B); the benchmark performance of, for instance, openhermes-2.5 and the newer GPT models thus has not been influenced by BioChatter development.",
 'The results of the benchmark provide useful criteria for selecting models and serve as a starting point for investigating variat

####  Paragraph 04

In [122]:
par0 = process_paragraph(orig_section_paragraphs[5])
print(par0)

We facilitate access to LLMs by enabling the use of both proprietary and open-source models, and we provide a flexible deployment framework for the latter. Proprietary models are currently the most economical solution for accessing state-of-the-art models and, as such, they are suitable for users just starting out or lacking the resources to deploy their own models. In contrast, open-source models are quickly catching up in terms of performance [@biollmbench] and are essential for the sustainability of the field [@doi:10.1038/d41586-024-00029-4]. We allow self-hosting of open-source models on any scale, from dedicated hardware with GPUs, to local deployment on end-user laptops, to browser-based deployment using web technology.


In [123]:
par1 = process_paragraph(mod_section_paragraphs[5])
print(par1)

We make it easier to use LLMs by allowing access to both proprietary and open-source models. Our framework offers a flexible deployment option for open-source models. Proprietary models are currently the most cost-effective choice for accessing the latest models and are ideal for beginners or those with limited resources. On the other hand, open-source models are rapidly improving in performance and are crucial for the long-term viability of the field (Biollmbench). Our platform enables users to host open-source models themselves, whether on dedicated hardware with GPUs, locally on their laptops, or through browser-based deployment using web technology.


In [124]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [125]:
display(paragraph_matches[-1])

('discussion',
 'We facilitate access to LLMs by enabling the use of both proprietary and open-source models, and we provide a flexible deployment framework for the latter. Proprietary models are currently the most economical solution for accessing state-of-the-art models and, as such, they are suitable for users just starting out or lacking the resources to deploy their own models. In contrast, open-source models are quickly catching up in terms of performance [@biollmbench] and are essential for the sustainability of the field [@doi:10.1038/d41586-024-00029-4]. We allow self-hosting of open-source models on any scale, from dedicated hardware with GPUs, to local deployment on end-user laptops, to browser-based deployment using web technology.',
 'We make it easier to use LLMs by allowing access to both proprietary and open-source models. Our framework offers a flexible deployment option for open-source models. Proprietary models are currently the most cost-effective choice for accessi

####  Paragraph 05

In [126]:
par0 = process_paragraph(orig_section_paragraphs[7])
print(par0)

The current generation of LLMs is not yet ready for unsupervised use in biomedical research with its vast array of unique subfields. Effectively supporting this diversity through robust and contextually aware LLM interactions is a daunting task. While we have taken steps to mitigate the risks of using LLMs such as independent benchmarks, fact-checking, and RAG processes, we cannot guarantee that the models will not produce harmful outputs. We see current LLMs, particularly in the scope of the BioCypher ecosystem, as helpful tools to assist human researchers, alleviating menial and repetitive tasks and helping with technical aspects such as query languages. They are not meant to replace human ingenuity and expertise but to augment it with their complementary strengths. Despite the user-friendly design of BioChatter, there may be a learning curve for researchers unfamiliar with LLMs or the specific functionalities of the framework. For maximising its benefit to the community, encouraging

In [127]:
par1 = process_paragraph(mod_section_paragraphs[7])
print(par1)

The current generation of Large Language Models (LLMs) is not yet suitable for unsupervised use in biomedical research due to the vast array of unique subfields within the field. Effectively supporting this diversity through robust and contextually aware interactions with LLMs is a challenging task. While efforts have been made to reduce risks associated with using LLMs, such as independent benchmarks, fact-checking, and Retrieval-Augmented Generation (RAG) processes, there is no guarantee that the models will not produce harmful outputs. Current LLMs, particularly within the BioCypher ecosystem, are viewed as tools to support human researchers by assisting with menial and repetitive tasks, as well as technical aspects like query languages. They are not intended to replace human ingenuity and expertise, but rather to enhance it with their complementary strengths. Despite the user-friendly design of BioChatter, researchers unfamiliar with LLMs or the specific functionalities of the fram

In [128]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [129]:
display(paragraph_matches[-1])

('discussion',
 'The current generation of LLMs is not yet ready for unsupervised use in biomedical research with its vast array of unique subfields. Effectively supporting this diversity through robust and contextually aware LLM interactions is a daunting task. While we have taken steps to mitigate the risks of using LLMs such as independent benchmarks, fact-checking, and RAG processes, we cannot guarantee that the models will not produce harmful outputs. We see current LLMs, particularly in the scope of the BioCypher ecosystem, as helpful tools to assist human researchers, alleviating menial and repetitive tasks and helping with technical aspects such as query languages. They are not meant to replace human ingenuity and expertise but to augment it with their complementary strengths. Despite the user-friendly design of BioChatter, there may be a learning curve for researchers unfamiliar with LLMs or the specific functionalities of the framework. For maximising its benefit to the commu

####  Paragraph 06

In [130]:
par0 = process_paragraph(orig_section_paragraphs[9])
print(par0)

Multitask learners that can synthesise, for instance, language, vision, and molecular measurements are an emerging field of research [@doi:10.48550/arXiv.2306.04529;@doi:10.48550/arXiv.2211.01786;@doi:10.48550/arXiv.2310.09478]. Autonomous agents for trivial tasks have already been developed on the basis of LLMs, and we expect this field to mature in the future [@doi:10.48550/arXiv.2308.11432]. As research on multimodal learning and agent behaviour progresses, we plan to integrate these developments into the BioChatter framework. All framework developments will be performed in light of the ethical implications of LLMs, and we will continue to support the use of open-source models to increase transparency and data privacy. While we focus on the biomedical field, the concept of our frameworks can easily be extended to other scientific domains by adjusting domain-specific prompts and data inputs, which are accessible in a composable and user-friendly manner in our frameworks [@biocypher].

In [131]:
par1 = process_paragraph(mod_section_paragraphs[9])
print(par1)

Emerging research shows that multitask learners, capable of synthesizing language, vision, and molecular measurements, are gaining traction [@doi:10.48550/arXiv.2306.04529;@doi:10.48550/arXiv.2211.01786;@doi:10.48550/arXiv.2310.09478]. Some autonomous agents for basic tasks have already been created using Large Language Models (LLMs), and we anticipate further advancements in this area [@doi:10.48550/arXiv.2308.11432]. As studies on multimodal learning and agent behavior advance, we aim to incorporate these findings into the BioChatter framework. Our framework development prioritizes ethical considerations related to LLMs, emphasizing the use of open-source models to enhance transparency and data privacy. Although our primary focus is on biomedicine, the adaptability of our frameworks allows for easy extension to other scientific fields through adjustments to domain-specific prompts and data inputs, which are conveniently accessible in our frameworks [@biocypher]. Our Python library is

In [132]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [133]:
display(paragraph_matches[-1])

('discussion',
 'Multitask learners that can synthesise, for instance, language, vision, and molecular measurements are an emerging field of research [@doi:10.48550/arXiv.2306.04529;@doi:10.48550/arXiv.2211.01786;@doi:10.48550/arXiv.2310.09478]. Autonomous agents for trivial tasks have already been developed on the basis of LLMs, and we expect this field to mature in the future [@doi:10.48550/arXiv.2308.11432]. As research on multimodal learning and agent behaviour progresses, we plan to integrate these developments into the BioChatter framework. All framework developments will be performed in light of the ethical implications of LLMs, and we will continue to support the use of open-source models to increase transparency and data privacy. While we focus on the biomedical field, the concept of our frameworks can easily be extended to other scientific domains by adjusting domain-specific prompts and data inputs, which are accessible in a composable and user-friendly manner in our framewo

## Methods

In [134]:
section_name = "methods"

In [135]:
pr_filename = pr_files[4].filename
assert section_name in pr_filename
print(pr_filename)

content/40.methods.md


### Original

In [136]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## (Supplementary / Online) Methods {.page_break_b


In [137]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

36

### Modified

In [138]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## (Supplementary / Online) Methods {.page_break_b


In [139]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

52

### Match

In [140]:
orig_section_paragraphs[0]

'## (Supplementary / Online) Methods {.page_break_before}'

In [141]:
mod_section_paragraphs[0]

'## (Supplementary / Online) Methods {.page_break_before}'

####  Paragraph 00

In [142]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

BioChatter (version 0.4.7 at the time of publication) is a Python library, supporting Python 3.10-3.12, which we ensure with a continuous integration pipeline on GitHub ([https://github.com/biocypher/biochatter](https://github.com/biocypher/biochatter)). We provide documentation at [https://biochatter.org](https://biochatter.org), including a tutorial and API reference. All packages are developed openly and according to modern standards of software development [@doi:10.1038/s41597-020-0486-7]; we use the permissive MIT licence to encourage downstream use and development. We include a code of conduct and contributor guidelines to offer accessibility and inclusivity to all who are interested in contributing to the framework.


In [143]:
par1 = (
    process_paragraph(mod_section_paragraphs[1:3])
    .replace("$$", "\n$$")
    .replace("\\text", "\n\\text")
)
print(par1)

BioChatter (version 0.4.7 at the time of publication) is a Python library that supports Python 3.10-3.12. The compatibility with these versions is ensured through a continuous integration pipeline on GitHub (https://github.com/biocypher/biochatter). Documentation for BioChatter can be found at https://biochatter.org, which includes a tutorial and API reference. The development of all packages is done openly and follows modern software development standards [@doi:10.1038/s41597-020-0486-7]. To encourage downstream use and development, we have chosen the permissive MIT license. Additionally, we have included a code of conduct and contributor guidelines to promote accessibility and inclusivity for all individuals interested in contributing to the framework. 
$$ 
\text{Equation (@id): Definition of important symbol} 
$$


In [144]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [145]:
display(paragraph_matches[-1])

('methods',
 'BioChatter (version 0.4.7 at the time of publication) is a Python library, supporting Python 3.10-3.12, which we ensure with a continuous integration pipeline on GitHub ([https://github.com/biocypher/biochatter](https://github.com/biocypher/biochatter)). We provide documentation at [https://biochatter.org](https://biochatter.org), including a tutorial and API reference. All packages are developed openly and according to modern standards of software development [@doi:10.1038/s41597-020-0486-7]; we use the permissive MIT licence to encourage downstream use and development. We include a code of conduct and contributor guidelines to offer accessibility and inclusivity to all who are interested in contributing to the framework.',
 'BioChatter (version 0.4.7 at the time of publication) is a Python library that supports Python 3.10-3.12. The compatibility with these versions is ensured through a continuous integration pipeline on GitHub (https://github.com/biocypher/biochatter).

####  Paragraph 01

In [146]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

BioChatter Light is a web app based on the Streamlit framework (version 1.31.1, [https://streamlit.io](https://streamlit.io)), which is written in Python and can be deployed locally or on a server ([https://github.com/biocypher/biochatter-light](https://github.com/biocypher/biochatter-light)). The ease with which Streamlit allows the creation of interactive web apps in pure Python enables rapid iteration and agile development of new features, with the tradeoff of limited customisation and scalability. This framework is suitable for rapid prototyping of bespoke solutions for specific use cases. For an up-to-date overview and preview of the current functionality of the platform, please visit the [online preview](https://chat.biocypher.org).


In [147]:
par1 = process_paragraph(
    mod_section_paragraphs[5]
)  # .replace("$$", "\n$$").replace("\\text", "\n\\text")
print(par1)

BioChatter Light is a web app based on the Streamlit framework (version 1.31.1, [https://streamlit.io](https://streamlit.io)), which is written in Python and can be deployed locally or on a server ([https://github.com/biocypher/biochatter-light](https://github.com/biocypher/biochatter-light)). The ease with which Streamlit allows the creation of interactive web apps in pure Python enables rapid iteration and agile development of new features, with the tradeoff of limited customization and scalability. This framework is suitable for rapid prototyping of bespoke solutions for specific use cases. For an up-to-date overview and preview of the current functionality of the platform, please visit the [online preview](https://chat.biocypher.org).


In [148]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [149]:
display(paragraph_matches[-1])

('methods',
 'BioChatter Light is a web app based on the Streamlit framework (version 1.31.1, [https://streamlit.io](https://streamlit.io)), which is written in Python and can be deployed locally or on a server ([https://github.com/biocypher/biochatter-light](https://github.com/biocypher/biochatter-light)). The ease with which Streamlit allows the creation of interactive web apps in pure Python enables rapid iteration and agile development of new features, with the tradeoff of limited customisation and scalability. This framework is suitable for rapid prototyping of bespoke solutions for specific use cases. For an up-to-date overview and preview of the current functionality of the platform, please visit the [online preview](https://chat.biocypher.org).',
 'BioChatter Light is a web app based on the Streamlit framework (version 1.31.1, [https://streamlit.io](https://streamlit.io)), which is written in Python and can be deployed locally or on a server ([https://github.com/biocypher/bioch

####  Paragraph 02

In [150]:
par0 = process_paragraph(orig_section_paragraphs[5])
print(par0)

BioChatter Next ([https://github.com/biocypher/biochatter-next](https://github.com/biocypher/biochatter-next)) is a modern web app with server-client architecture, based on the open-source template of ChatGPT-Next-Web ([https://github.com/ChatGPTNextWeb/ChatGPT-Next-Web](https://github.com/ChatGPTNextWeb/ChatGPT-Next-Web)). It is written combining Typescript and Python and uses Next.js (v13.4.9) for a sleek frontend and Flask (v3.0.0) as backend. It demonstrates the use of BioChatter in a modern web app, including full customisation and scalability and localisation in 18 languages. However, this comes at the cost of increased complexity and development time. To provide seamless integration of the BioChatter backend into existing frontend solutions, we provide the server implementation at [https://github.com/biocypher/biochatter-server](https://github.com/biocypher/biochatter-server) and as a Docker image in our Docker Hub organisation ([https://hub.docker.com/repository/docker/biocyphe

In [151]:
par1 = process_paragraph(
    mod_section_paragraphs[6]
)  # .replace("$$", "\n$$").replace("\\text", "\n\\text")
print(par1)

BioChatter Next ([https://github.com/biocypher/biochatter-next](https://github.com/biocypher/biochatter-next)) is a contemporary web application with a server-client architecture, built upon the open-source framework of ChatGPT-Next-Web ([https://github.com/ChatGPTNextWeb/ChatGPT-Next-Web](https://github.com/ChatGPTNextWeb/ChatGPT-Next-Web)). It is developed using a combination of Typescript and Python, utilizing Next.js (v13.4.9) for the frontend and Flask (v3.0.0) for the backend. The platform showcases the application of BioChatter within a modern web environment, offering extensive customization, scalability, and support for localization in 18 languages. Nevertheless, this enhanced functionality is accompanied by increased complexity and longer development cycles. To facilitate the seamless integration of the BioChatter backend with existing frontend solutions, we offer the server implementation at [https://github.com/biocypher/biochatter-server](https://github.com/biocypher/biocha

In [152]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [153]:
display(paragraph_matches[-1])

('methods',
 'BioChatter Next ([https://github.com/biocypher/biochatter-next](https://github.com/biocypher/biochatter-next)) is a modern web app with server-client architecture, based on the open-source template of ChatGPT-Next-Web ([https://github.com/ChatGPTNextWeb/ChatGPT-Next-Web](https://github.com/ChatGPTNextWeb/ChatGPT-Next-Web)). It is written combining Typescript and Python and uses Next.js (v13.4.9) for a sleek frontend and Flask (v3.0.0) as backend. It demonstrates the use of BioChatter in a modern web app, including full customisation and scalability and localisation in 18 languages. However, this comes at the cost of increased complexity and development time. To provide seamless integration of the BioChatter backend into existing frontend solutions, we provide the server implementation at [https://github.com/biocypher/biochatter-server](https://github.com/biocypher/biochatter-server) and as a Docker image in our Docker Hub organisation ([https://hub.docker.com/repository/d

####  Paragraph 03

In [154]:
par0 = process_paragraph(orig_section_paragraphs[8])
print(par0)

The benchmarking framework examines a matrix of component combinations using the parameterisation feature of Pytest [@pytest]. This implementation allows for the automated evaluation of all possible combinations of components, such as LLMs, prompts, and datasets. We performed the benchmarks on a MacBook Pro with an M3 Max chip with 40-core GPU and 128GB of RAM. As a default, we ran each test five times to account for the stochastic nature of LLMs. We generally set the temperature to the lowest value possible for each model to decrease fluctuation.


In [155]:
par1 = process_paragraph(mod_section_paragraphs[9:11]).replace(
    "$$", "\n$$"
)  # .replace("\\text\{LLM", "\n\\text\{LLM")
print(par1)

The benchmarking framework examines a matrix of component combinations using the parameterization feature of Pytest [@pytest]. This implementation allows for the automated evaluation of all possible combinations of components, such as Large Language Models (LLMs), prompts, and datasets. We performed the benchmarks on a MacBook Pro with an M3 Max chip with a 40-core GPU and 128GB of RAM. As a default, we ran each test five times to account for the stochastic nature of LLMs. We generally set the temperature to the lowest value possible for each model to decrease fluctuation. 
$$ \text{Symbols:}\\ \text{LLMs} - \text{Large Language Models} 
$$


In [156]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [157]:
display(paragraph_matches[-1])

('methods',
 'The benchmarking framework examines a matrix of component combinations using the parameterisation feature of Pytest [@pytest]. This implementation allows for the automated evaluation of all possible combinations of components, such as LLMs, prompts, and datasets. We performed the benchmarks on a MacBook Pro with an M3 Max chip with 40-core GPU and 128GB of RAM. As a default, we ran each test five times to account for the stochastic nature of LLMs. We generally set the temperature to the lowest value possible for each model to decrease fluctuation.',
 'The benchmarking framework examines a matrix of component combinations using the parameterization feature of Pytest [@pytest]. This implementation allows for the automated evaluation of all possible combinations of components, such as Large Language Models (LLMs), prompts, and datasets. We performed the benchmarks on a MacBook Pro with an M3 Max chip with a 40-core GPU and 128GB of RAM. As a default, we ran each test five ti

####  Paragraph 04

In [158]:
par0 = process_paragraph(orig_section_paragraphs[9:19]).replace(" - ", "\n- ")
print(par0)

The Pytest matrix uses a hash-based system to evaluate whether a model-dataset combination has been run before. Briefly, the hash is calculated from the dictionary representation of the test parameters, and the test is skipped if the combination of hash and model name is already present in the database. This hashing optimises for efficiency by only running modified or newly added tests. The individual dimensions of the matrix are:
- **LLMs**: Testing proprietary (OpenAI) and open-source models (commonly using the Xorbits Inference API and HuggingFace models) against the same set of tasks is the primary aim of our benchmarking framework. We facilitate the automation of testing by including a programmatic way of deploying open-source models.
- **prompts**: Since model performance can dramatically rely on the used prompts, a set of prompts for each task with varying degrees of specificity and fixed as well as variable components is used to evaluate this variability.
- **datasets**: We tes

In [159]:
par1 = process_paragraph(mod_section_paragraphs[11:21]).replace(" - ", "\n- ")
print(par1)

The Pytest matrix utilizes a hash-based system to determine if a model-dataset combination has been previously executed. In essence, the hash is computed from the dictionary representation of the test parameters, and the test is skipped if the hash combined with the model name already exists in the database. This hashing strategy optimizes efficiency by only executing modified or newly added tests. The key dimensions of the matrix include:
- **LLMs**: The primary objective of our benchmarking framework is to test both proprietary (OpenAI) and open-source models (commonly leveraging the Xorbits Inference API and HuggingFace models) against the same set of tasks. We streamline the testing process by incorporating a programmatic approach to deploying open-source models.
- **prompts**: Given that model performance can significantly depend on the prompts used, we employ a range of prompts for each task with varying levels of specificity and components that are fixed or variable to assess th

In [160]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [161]:
display(paragraph_matches[-1])

('methods',
 'The Pytest matrix uses a hash-based system to evaluate whether a model-dataset combination has been run before. Briefly, the hash is calculated from the dictionary representation of the test parameters, and the test is skipped if the combination of hash and model name is already present in the database. This hashing optimises for efficiency by only running modified or newly added tests. The individual dimensions of the matrix are:\n- **LLMs**: Testing proprietary (OpenAI) and open-source models (commonly using the Xorbits Inference API and HuggingFace models) against the same set of tasks is the primary aim of our benchmarking framework. We facilitate the automation of testing by including a programmatic way of deploying open-source models.\n- **prompts**: Since model performance can dramatically rely on the used prompts, a set of prompts for each task with varying degrees of specificity and fixed as well as variable components is used to evaluate this variability.\n- **d

####  Paragraph 05

In [162]:
par0 = process_paragraph(orig_section_paragraphs[20])
print(par0)

To prevent leakage of benchmarking data (and subsequent contamination of future LLMs), we implement an encryption routine on the benchmark datasets. The encryption is performed using a hybrid encryption scheme, where the data are encrypted with a symmetric key, which is in turn encrypted with an asymmetric key. The datasets are stored in a dedicated encrypted pipeline that is only accessible to the workflow that executes the benchmark. These processes are implemented at [https://github.com/biocypher/llm-test-dataset](https://github.com/biocypher/llm-test-dataset) and accessed from the benchmark procedure in BioChatter.


In [163]:
par1 = process_paragraph(mod_section_paragraphs[24])
print(par1)

To prevent leakage of benchmarking data (and subsequent contamination of future LLMs), an encryption routine is implemented on the benchmark datasets. The encryption is carried out using a hybrid encryption scheme, where the data are encrypted with a symmetric key, which is then encrypted with an asymmetric key. The datasets are stored in a dedicated encrypted pipeline that is exclusively accessible to the workflow executing the benchmark. These processes are implemented at [https://github.com/biocypher/llm-test-dataset](https://github.com/biocypher/llm-test-dataset) and accessed from the benchmark procedure in BioChatter.


In [164]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [165]:
display(paragraph_matches[-1])

('methods',
 'To prevent leakage of benchmarking data (and subsequent contamination of future LLMs), we implement an encryption routine on the benchmark datasets. The encryption is performed using a hybrid encryption scheme, where the data are encrypted with a symmetric key, which is in turn encrypted with an asymmetric key. The datasets are stored in a dedicated encrypted pipeline that is only accessible to the workflow that executes the benchmark. These processes are implemented at [https://github.com/biocypher/llm-test-dataset](https://github.com/biocypher/llm-test-dataset) and accessed from the benchmark procedure in BioChatter.',
 'To prevent leakage of benchmarking data (and subsequent contamination of future LLMs), an encryption routine is implemented on the benchmark datasets. The encryption is carried out using a hybrid encryption scheme, where the data are encrypted with a symmetric key, which is then encrypted with an asymmetric key. The datasets are stored in a dedicated en

####  Paragraph 06

In [166]:
par0 = process_paragraph(orig_section_paragraphs[22])
print(par0)

We utilise the close connection between BioChatter and the BioCypher framework [@biocypher] to integrate knowledge graph (KG) queries into the BioChatter API. In the BioCypher KG creation, we use a configuration file to map KG contents to ontology terms, including information about each of the entities. For instance, we detail the properties of a node and the source and target classes of an edge. Additionally, during the KG build process, we enrich this information and save it to a YAML file and, optionally, directly to the KG. This information is used by BioChatter to tune its understanding of the KG, which allows the LLM to query the KG more efficiently.


In [167]:
par1 = process_paragraph(mod_section_paragraphs[26])
print(par1)

We utilize the close connection between BioChatter and the BioCypher framework (Johnson et al., 2020) to integrate knowledge graph (KG) queries into the BioChatter API. In the BioCypher KG creation, we use a configuration file to map KG contents to ontology terms, including information about each of the entities. For instance, we detail the properties of a node and the source and target classes of an edge. Additionally, during the KG build process, we enrich this information and save it to a YAML file and, optionally, directly to the KG. This information is used by BioChatter to tune its understanding of the KG, which allows the Large Language Model (LLM) to query the KG more efficiently.


In [168]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [169]:
display(paragraph_matches[-1])

('methods',
 'We utilise the close connection between BioChatter and the BioCypher framework [@biocypher] to integrate knowledge graph (KG) queries into the BioChatter API. In the BioCypher KG creation, we use a configuration file to map KG contents to ontology terms, including information about each of the entities. For instance, we detail the properties of a node and the source and target classes of an edge. Additionally, during the KG build process, we enrich this information and save it to a YAML file and, optionally, directly to the KG. This information is used by BioChatter to tune its understanding of the KG, which allows the LLM to query the KG more efficiently.',
 'We utilize the close connection between BioChatter and the BioCypher framework (Johnson et al., 2020) to integrate knowledge graph (KG) queries into the BioChatter API. In the BioCypher KG creation, we use a configuration file to map KG contents to ontology terms, including information about each of the entities. Fo

####  Paragraph 07

In [170]:
par0 = process_paragraph(orig_section_paragraphs[23])
print(par0)

By understanding the context of the KG, the exact contents, and the exact spelling of all identifiers and properties, we effectively support the LLM in generating correct queries. The query generation process is broken up into multiple steps by BioChatter: recognising entities and relationships according to the user's question, estimating properties to be used in the query, and generating a syntactically correct query in the query language of the database, based on the results from the previous steps and constraints given by the KG schema information. This procedure is implemented in the `prompts.py` module. To evaluate the quality of this process, we dedicate a module in the benchmark to the query generation process with a range of questions and KG schemata.


In [171]:
par1 = (
    process_paragraph(mod_section_paragraphs[27:28])
    .replace("$$", "\n$$")
    .replace("\\text", "\n\\text")
    + "\n"
    + process_paragraph(mod_section_paragraphs[28:31])
)
print(par1)


$$ 
\text{BioCypher KG creation process:} 
$$ {#kg_creation}
The BioCypher KG creation process involves using a configuration file to map KG contents to ontology terms. This includes detailing the properties of a node and the source and target classes of an edge. The enriched information is saved to a YAML file and, if desired, directly to the KG. This information is crucial for BioChatter to optimize its understanding of the KG, enabling the LLM to query the KG more effectively. By understanding the context of the knowledge graph (KG), the exact contents, and the exact spelling of all identifiers and properties, we effectively support the Large Language Model (LLM) in generating correct queries. The query generation process is broken up into multiple steps by BioChatter: recognizing entities and relationships according to the user's question, estimating properties to be used in the query, and generating a syntactically correct query in the query language of the database, based on the

In [172]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [173]:
display(paragraph_matches[-1])

('methods',
 "By understanding the context of the KG, the exact contents, and the exact spelling of all identifiers and properties, we effectively support the LLM in generating correct queries. The query generation process is broken up into multiple steps by BioChatter: recognising entities and relationships according to the user's question, estimating properties to be used in the query, and generating a syntactically correct query in the query language of the database, based on the results from the previous steps and constraints given by the KG schema information. This procedure is implemented in the `prompts.py` module. To evaluate the quality of this process, we dedicate a module in the benchmark to the query generation process with a range of questions and KG schemata.",
 "\n$$ \n\\text{BioCypher KG creation process:} \n$$ {#kg_creation}\nThe BioCypher KG creation process involves using a configuration file to map KG contents to ontology terms. This includes detailing the propertie

####  Paragraph 08

In [174]:
par0 = process_paragraph(orig_section_paragraphs[24])
print(par0)

To illustrate the usage of this feature, we provide a demonstration repository at [https://github.com/biocypher/pole](https://github.com/biocypher/pole) including a KG build procedure and an instance of BioChatter Light, which can be run using a single Docker Compose command. The pole KG can also be used in conjunction with the BioChatter Next app by using the `docker-compose.yaml` file to build the application locally. A demonstration of this use case is available in [Supplementary Note 1: Knowledge Graph Retrieval-Augmented Generation] and on our website ([https://biochatter.org/vignette-kg/](https://biochatter.org/vignette-kg/)).


In [175]:
par1 = process_paragraph(
    [
        process_paragraph(mod_section_paragraphs[31:32])
        .replace("$$", "\n$$")
        .replace("\\text", "\n\\text"),
        process_paragraph(mod_section_paragraphs[32:33]),
    ]
)
print(par1)

This evaluation process helps us understand how well the LLM can generate queries based on the information provided by the KG and the user's input. It allows us to assess the accuracy and efficiency of the query generation process in different scenarios and with various types of KG schemata. To demonstrate the utility of this feature, we have created a demonstration repository available at [https://github.com/biocypher/pole](https://github.com/biocypher/pole). This repository includes a procedure for building a knowledge graph (KG) and an instance of BioChatter Light, which can be easily run using a single Docker Compose command. Furthermore, the pole KG can be utilized in combination with the BioChatter Next application by utilizing the `docker-compose.yaml` file to locally build the application. A demonstration showcasing this use case can be found in [Supplementary Note 1: Knowledge Graph Retrieval-Augmented Generation] and is also accessible on our website at [https://biochatter.or

In [176]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [177]:
display(paragraph_matches[-1])

('methods',
 'To illustrate the usage of this feature, we provide a demonstration repository at [https://github.com/biocypher/pole](https://github.com/biocypher/pole) including a KG build procedure and an instance of BioChatter Light, which can be run using a single Docker Compose command. The pole KG can also be used in conjunction with the BioChatter Next app by using the `docker-compose.yaml` file to build the application locally. A demonstration of this use case is available in [Supplementary Note 1: Knowledge Graph Retrieval-Augmented Generation] and on our website ([https://biochatter.org/vignette-kg/](https://biochatter.org/vignette-kg/)).',
 "This evaluation process helps us understand how well the LLM can generate queries based on the information provided by the KG and the user's input. It allows us to assess the accuracy and efficiency of the query generation process in different scenarios and with various types of KG schemata. To demonstrate the utility of this feature, we h

####  Paragraph 09

In [178]:
par0 = process_paragraph(orig_section_paragraphs[26])
print(par0)

While current LLMs possess extensive internal general knowledge, they may not know how to prioritise very specific scientific results, or they may not have had access to some research articles in their training data (e.g., due to their recency or licensing issues). To bridge this gap, we can provide additional information from relevant publications to the model via the prompt. However, we frequently cannot add entire publications to the prompt, since the input length of current models is still restricted; we need to isolate the information that is specifically relevant to the question given by the user. To find this information, we perform a semantic similarity search between the user’s question and the contents of user-provided scientific articles (or other texts). The most efficient way to do this mapping is by using a vector database [@doi:10.48550/arxiv.2308.07107].


In [179]:
par1 = process_paragraph(
    [
        process_paragraph(
            mod_section_paragraphs[34:35]
        ),  # .replace("$$", "\n$$").replace("\\text", "\n\\text"),
        # process_paragraph(mod_section_paragraphs[32:33]),
    ]
)
print(par1)

While current Large Language Models (LLMs) possess extensive internal general knowledge, they may not prioritize very specific scientific results or have access to all research articles in their training data, possibly due to recency or licensing issues. To address this limitation, we can enhance the model's knowledge by providing additional information from relevant publications through the prompt. However, due to the current input length restrictions of LLMs, we cannot include entire publications in the prompt. Therefore, we must identify and isolate the information that is directly pertinent to the user's query. To achieve this, we conduct a semantic similarity search between the user's question and the content of user-provided scientific articles or other texts. Utilizing a vector database is the most effective method for this mapping process [@doi:10.48550/arxiv.2308.07107].


In [180]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [181]:
display(paragraph_matches[-1])

('methods',
 'While current LLMs possess extensive internal general knowledge, they may not know how to prioritise very specific scientific results, or they may not have had access to some research articles in their training data (e.g., due to their recency or licensing issues). To bridge this gap, we can provide additional information from relevant publications to the model via the prompt. However, we frequently cannot add entire publications to the prompt, since the input length of current models is still restricted; we need to isolate the information that is specifically relevant to the question given by the user. To find this information, we perform a semantic similarity search between the user’s question and the contents of user-provided scientific articles (or other texts). The most efficient way to do this mapping is by using a vector database [@doi:10.48550/arxiv.2308.07107].',
 "While current Large Language Models (LLMs) possess extensive internal general knowledge, they may n

####  Paragraph 10

In [182]:
par0 = process_paragraph(orig_section_paragraphs[27])
print(par0)

The contextual background information provided by the user (e.g., by uploading a scientific article of prior work related to the experiment to be interpreted) is split into pieces suitable to be digested by the LLM, which are individually embedded by the model. These embeddings (represented by vectors) are used to store the text fragments in a vector database; the storage as vectors allows fast and efficient retrieval of similar entities via the comparison of individual vectors. For example, the two sentences “Amyloid beta levels are associated with Alzheimer’s Disease stage.” and “One of the most important clinical markers of AD progression is the amount of deposited A-beta 42.” would be closely associated in a vector database (given the embedding model is of sufficient quality, i.e., similar to GPT-3 or better), while traditional text-based similarity metrics probably would not identify them as highly similar.


In [183]:
par1 = (
    process_paragraph(
        [
            process_paragraph(mod_section_paragraphs[35:36]),
            process_paragraph(mod_section_paragraphs[36:37]),
        ]
    )
    .replace("Model $$", "Model\n$$")
    .replace("LLM:", "\nLLM:")
    .replace("The contextual", "\nThe contextual")
)
print(par1)

$$ 
LLM: Large Language Model
$$ 
The contextual background information provided by the user, such as a scientific article related to the experiment to be interpreted, is divided into segments suitable for processing by the Large Language Model (LLM). Each segment is individually embedded by the model. These embeddings, represented as vectors, are utilized to store the text fragments in a vector database. Storing the information as vectors enables rapid and efficient retrieval of similar entities through the comparison of individual vectors. For instance, the sentences "Amyloid beta levels are associated with Alzheimer's Disease stage." and "One of the most crucial clinical markers of AD progression is the amount of deposited A-beta 42." would be closely linked in a vector database if the embedding model is of high quality, similar to GPT-3 or superior. In contrast, traditional text-based similarity metrics might not recognize them as highly similar.


In [184]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [185]:
display(paragraph_matches[-1])

('methods',
 'The contextual background information provided by the user (e.g., by uploading a scientific article of prior work related to the experiment to be interpreted) is split into pieces suitable to be digested by the LLM, which are individually embedded by the model. These embeddings (represented by vectors) are used to store the text fragments in a vector database; the storage as vectors allows fast and efficient retrieval of similar entities via the comparison of individual vectors. For example, the two sentences “Amyloid beta levels are associated with Alzheimer’s Disease stage.” and “One of the most important clinical markers of AD progression is the amount of deposited A-beta 42.” would be closely associated in a vector database (given the embedding model is of sufficient quality, i.e., similar to GPT-3 or better), while traditional text-based similarity metrics probably would not identify them as highly similar.',
 '$$ \nLLM: Large Language Model\n$$ \nThe contextual back

####  Paragraph 11

In [186]:
par0 = process_paragraph(orig_section_paragraphs[28])
print(par0)

By comparing the user’s question to prior knowledge in the vector database, we can extract the relevant pieces of information from the entire background. Even better, we can first use an LLM to generate an answer to the user's question and then use this answer to query the vector database for relevant information. Regardless of whether the initial answer is correct, it is likely that the "fake answer" is more semantically similar to the relevant pieces of information than the user's question [@doi:10.48550/arXiv.2308.07107]. Semantic search results (for instance, single sentences directly related to the topic of the question) are then sufficiently small to be added to the prompt. In this way, the model can learn from additional context without the need for retraining or fine-tuning. This method is sometimes described as in-context learning [@doi:10.48550/arxiv.2303.17580] or retrieval-augmented generation [@rag].


In [187]:
par1 = process_paragraph(mod_section_paragraphs[38])
print(par1)

By comparing the user’s question to prior knowledge in the vector database, we can extract the relevant pieces of information from the entire background. Even better, we can first use a Large Language Model (LLM) to generate an answer to the user's question and then use this answer to query the vector database for relevant information. Regardless of whether the initial answer is correct, it is likely that the "fake answer" is more semantically similar to the relevant pieces of information than the user's question [@doi:10.48550/arXiv.2308.07107]. Semantic search results (for instance, single sentences directly related to the topic of the question) are then sufficiently small to be added to the prompt. In this way, the model can learn from additional context without the need for retraining or fine-tuning. This method is sometimes described as in-context learning [@doi:10.48550/arxiv.2303.17580] or retrieval-augmented generation [@rag].


In [188]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [189]:
display(paragraph_matches[-1])

('methods',
 'By comparing the user’s question to prior knowledge in the vector database, we can extract the relevant pieces of information from the entire background. Even better, we can first use an LLM to generate an answer to the user\'s question and then use this answer to query the vector database for relevant information. Regardless of whether the initial answer is correct, it is likely that the "fake answer" is more semantically similar to the relevant pieces of information than the user\'s question [@doi:10.48550/arXiv.2308.07107]. Semantic search results (for instance, single sentences directly related to the topic of the question) are then sufficiently small to be added to the prompt. In this way, the model can learn from additional context without the need for retraining or fine-tuning. This method is sometimes described as in-context learning [@doi:10.48550/arxiv.2303.17580] or retrieval-augmented generation [@rag].',
 'By comparing the user’s question to prior knowledge i

####  Paragraph 12

In [190]:
par0 = process_paragraph(orig_section_paragraphs[29])
print(par0)

To provide access to this functionality in BioChatter, we implement classes for the connection to, and management of, vector database systems (in the `vectorstore.py` module), and for performing semantic search on the vector database and injecting the results into the prompt (in the `vectorstore_agent.py` module). An analogous implementation for KG retrieval is available in the `database_agent.py` module. Both retrieval mechanisms are integrated and provided to the BioChatter API via the `rag_agent.py` module. To demonstrate the use of the API, we add a “Retrieval-Augmented Generation” tab to the preview apps that allows the upload of text documents to be added to a vector database, which then can be queried to add contextual information to the prompt sent to the primary model. This contextual information is transparently displayed. Since this functionality requires a connection to a vector database system, we provide connectivity to a Milvus service, including a way to start the servi

In [191]:
par1 = process_paragraph(mod_section_paragraphs[40:43])
print(par1)

To provide access to this functionality in BioChatter, we implement classes for the connection to, and management of, vector database systems (in the `vectorstore.py` module), and for performing semantic search on the vector database and injecting the results into the prompt (in the `vectorstore_agent.py` module). An analogous implementation for Knowledge Graph (KG) retrieval is available in the `database_agent.py` module. Both retrieval mechanisms are integrated and provided to the BioChatter API via the `rag_agent.py` module. To demonstrate the use of the API, we add a “Retrieval-Augmented Generation” tab to the preview apps that allows the upload of text documents to be added to a vector database, which then can be queried to add contextual information to the prompt sent to the primary model. This contextual information is transparently displayed. Since this functionality requires a connection to a vector database system, we provide connectivity to a Milvus service, including a way 

In [192]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [193]:
display(paragraph_matches[-1])

('methods',
 'To provide access to this functionality in BioChatter, we implement classes for the connection to, and management of, vector database systems (in the `vectorstore.py` module), and for performing semantic search on the vector database and injecting the results into the prompt (in the `vectorstore_agent.py` module). An analogous implementation for KG retrieval is available in the `database_agent.py` module. Both retrieval mechanisms are integrated and provided to the BioChatter API via the `rag_agent.py` module. To demonstrate the use of the API, we add a “Retrieval-Augmented Generation” tab to the preview apps that allows the upload of text documents to be added to a vector database, which then can be queried to add contextual information to the prompt sent to the primary model. This contextual information is transparently displayed. Since this functionality requires a connection to a vector database system, we provide connectivity to a Milvus service, including a way to s

####  Paragraph 13

In [194]:
par0 = process_paragraph(orig_section_paragraphs[32])
print(par0)

To facilitate access to open-source models, we adopt a flexible deployment framework based on the Xorbits Inference API [@{https://github.com/xorbitsai/inference}]. Xorbits Inference includes a large number of open-source models out of the box, and new models from Hugging Face Hub [@{https://huggingface.co/}] can be added using the intuitive graphical user interface. We used Xorbits Inference version 0.8.4 to deploy the benchmarked models, and we provide a Docker Compose repository to deploy the app on a Linux server with Nvidia GPUs ([https://github.com/biocypher/xinference-docker-builtin/](https://github.com/biocypher/xinference-docker-builtin/)). This Compose uses the multi-architecture image (for ARM64 and AMD64 chips) we provide on our Docker Hub organisation ([https://hub.docker.com/repository/docker/biocypher/xinference-builtin](https://hub.docker.com/repository/docker/biocypher/xinference-builtin)). On Mac OS with Apple Silicon chips, Docker does not have access to the GPU driv

In [195]:
par1 = (
    process_paragraph(
        [
            process_paragraph(mod_section_paragraphs[45:46]),
            mod_section_paragraphs[46],
            process_paragraph(mod_section_paragraphs[47]),
        ]
    )
    .replace(" $$ X", "\n\n$$\nX")
    .replace("$$ {", "\n$$ {")
    .replace("On Mac OS", "\n\nOn Mac OS")
)
print(par1)

To facilitate access to open-source models, we adopted a flexible deployment framework based on the Xorbits Inference API [@{https://github.com/xorbitsai/inference}]. Xorbits Inference includes a large number of open-source models out of the box, and new models from Hugging Face Hub [@{https://huggingface.co/}] can be added using the intuitive graphical user interface. We utilized Xorbits Inference version 0.8.4 to deploy the benchmarked models, and we provide a Docker Compose repository to deploy the app on a Linux server with Nvidia GPUs ([https://github.com/biocypher/xinference-docker-builtin/](https://github.com/biocypher/xinference-docker-builtin/)). This Compose uses the multi-architecture image (for ARM64 and AMD64 chips) we provide on our Docker Hub organization ([https://hub.docker.com/repository/docker/biocypher/xinference-builtin](https://hub.docker.com/repository/docker/biocypher/xinference-builtin)).

$$
X = Y + Z 
$$ {#eq1} 

On Mac OS with Apple Silicon chips, Docker doe

In [196]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [197]:
display(paragraph_matches[-1])

('methods',
 'To facilitate access to open-source models, we adopt a flexible deployment framework based on the Xorbits Inference API [@{https://github.com/xorbitsai/inference}]. Xorbits Inference includes a large number of open-source models out of the box, and new models from Hugging Face Hub [@{https://huggingface.co/}] can be added using the intuitive graphical user interface. We used Xorbits Inference version 0.8.4 to deploy the benchmarked models, and we provide a Docker Compose repository to deploy the app on a Linux server with Nvidia GPUs ([https://github.com/biocypher/xinference-docker-builtin/](https://github.com/biocypher/xinference-docker-builtin/)). This Compose uses the multi-architecture image (for ARM64 and AMD64 chips) we provide on our Docker Hub organisation ([https://hub.docker.com/repository/docker/biocypher/xinference-builtin](https://hub.docker.com/repository/docker/biocypher/xinference-builtin)). On Mac OS with Apple Silicon chips, Docker does not have access t

####  Paragraph 14

In [198]:
par0 = process_paragraph(orig_section_paragraphs[34])
print(par0)

The ability of LLMs to control external software, including other LLMs, opens up a wide range of possibilities for the orchestration of complex tasks. A simple example is the implementation of a correcting agent, which receives the output of the primary model and checks it for factual correctness. If the agent detects an error, it can prompt the primary model to correct its output, or forward this correction to the user directly. Since this relies on the internal knowledge base of the correcting agent, the same caveats apply, as the correcting agent may confabulate as well. However, since the agent is independent of the primary model (being set up with dedicated prompts), it is less likely to confabulate in the same way.


In [199]:
par1 = process_paragraph(
    [
        process_paragraph(mod_section_paragraphs[49]),
        # mod_section_paragraphs[46],
        # process_paragraph(mod_section_paragraphs[47])
    ]
)  # .replace(" $$ X", "\n\n$$\nX").replace("$$ {", "\n$$ {").replace("On Mac OS", "\n\nOn Mac OS")
print(par1)

The capability of Large Language Models (LLMs) to interact with external software, including other LLMs, presents a multitude of opportunities for orchestrating intricate tasks. An illustrative instance involves the deployment of a correction agent, which receives the primary model's output and verifies its factual accuracy. In case an error is identified, the agent can either prompt the primary model to rectify its output or directly convey the correction to the user. This process is contingent upon the internal knowledge repository of the correction agent, thereby subject to similar limitations, as the agent may also produce incorrect information. Nevertheless, due to the agent's autonomy from the primary model (established with specific prompts), it is less susceptible to generating erroneous data in the same manner.


In [200]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [201]:
display(paragraph_matches[-1])

('methods',
 'The ability of LLMs to control external software, including other LLMs, opens up a wide range of possibilities for the orchestration of complex tasks. A simple example is the implementation of a correcting agent, which receives the output of the primary model and checks it for factual correctness. If the agent detects an error, it can prompt the primary model to correct its output, or forward this correction to the user directly. Since this relies on the internal knowledge base of the correcting agent, the same caveats apply, as the correcting agent may confabulate as well. However, since the agent is independent of the primary model (being set up with dedicated prompts), it is less likely to confabulate in the same way.',
 "The capability of Large Language Models (LLMs) to interact with external software, including other LLMs, presents a multitude of opportunities for orchestrating intricate tasks. An illustrative instance involves the deployment of a correction agent, w

####  Paragraph 15

In [202]:
par0 = process_paragraph(orig_section_paragraphs[35])
print(par0)

This approach can be extended to a more complex model chain, where the correcting agent, for example, can query a knowledge graph or a vector database to ground its responses in prior knowledge. These chains are easy to implement, and some are available out of the box in the LangChain framework [@langchain]. However, they can behave unpredictably, which increases with the number of links in the chain and, as such, should be tightly controlled. They also add to the computational burden of the system, which is particularly relevant for deployments on end-user devices.


In [203]:
par1 = process_paragraph(
    [
        process_paragraph(mod_section_paragraphs[51]),
        # mod_section_paragraphs[46],
        # process_paragraph(mod_section_paragraphs[47])
    ]
)  # .replace(" $$ X", "\n\n$$\nX").replace("$$ {", "\n$$ {").replace("On Mac OS", "\n\nOn Mac OS")
print(par1)

This approach can be extended to a more complex model chain, where the correcting agent, for example, can query a knowledge graph or a vector database to ground its responses in prior knowledge. These chains are easy to implement, and some are available out of the box in the LangChain framework (Doe et al., 2020). However, they can behave unpredictably, which increases with the number of links in the chain and, as such, should be tightly controlled. They also add to the computational burden of the system, which is particularly relevant for deployments on end-user devices.


In [204]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [205]:
display(paragraph_matches[-1])

('methods',
 'This approach can be extended to a more complex model chain, where the correcting agent, for example, can query a knowledge graph or a vector database to ground its responses in prior knowledge. These chains are easy to implement, and some are available out of the box in the LangChain framework [@langchain]. However, they can behave unpredictably, which increases with the number of links in the chain and, as such, should be tightly controlled. They also add to the computational burden of the system, which is particularly relevant for deployments on end-user devices.',
 'This approach can be extended to a more complex model chain, where the correcting agent, for example, can query a knowledge graph or a vector database to ground its responses in prior knowledge. These chains are easy to implement, and some are available out of the box in the LangChain framework (Doe et al., 2020). However, they can behave unpredictably, which increases with the number of links in the chain

# Close connections

In [206]:
g.close()

# Save

In [207]:
len(paragraph_matches)

37

In [208]:
paragraph_matches[:2]

[('abstract',
  'Current-generation Large Language Models (LLMs) have stirred enormous interest in recent months, yielding great potential for accessibility and automation, while simultaneously posing significant challenges and risk of misuse. To facilitate interfacing with LLMs in the biomedical space, while at the same time safeguarding their functionalities through sensible constraints, we propose a dedicated, open-source framework: BioChatter. Based on open-source software packages, we synergise the many functionalities that are currently developing around LLMs, such as knowledge integration / retrieval-augmented generation, model chaining, and benchmarking, resulting in an easy-to-use and inclusive framework for application in many use cases of biomedicine. We focus on robust and user-friendly implementation, including ways to deploy privacy-preserving local open-source LLMs. We demonstrate use cases via two multi-purpose web apps ([https://chat.biocypher.org](https://chat.biocyph

In [209]:
df = pd.DataFrame(paragraph_matches).rename(
    columns={
        0: "section",
        1: "original",
        2: "modified",
    }
)

In [210]:
df.shape

(37, 3)

In [211]:
df.head()

,section,original,modified
0,abstract,Current-generation Large Language Models (LLMs...,Large Language Models (LLMs) have generated si...
1,introduction,"Despite technological advances, understanding ...","Despite technological advances, understanding ..."
2,introduction,Large Language Models (LLMs) of the current ge...,The latest generation of Large Language Models...
3,introduction,Computational biomedicine involves many tasks ...,Computational biomedicine encompasses various ...
4,results,The framework is designed to be modular: any o...,"The framework is designed to be modular, allow..."


In [212]:
df.to_pickle(OUTPUT_FILE_PATH)